# Orbits of bitangents on symmetric quartics

## Table of contents
1. [Introduction](#introduction)
2. [Functions](#functions)
3. [Type I: The Klein ($\text{PSL}_2(7)$) quartic](#klein)
4. [Type II: The Clebsch](#fermat)
5. [Type III: The (48,33) quartic](#type3)
6. [Type IV: The $S_4$ quartic](#S4)
7. [Type V: The (16,13) quartic](#1613)
8. [Type VI: The $C_9$ quartic](#C9)
9. [Type VII: The $D_8$ quartic](#D8)
10. [Type IX: The $S_3$ quartic](#S3)
11. [Type X: The $K_4$ quartic](#K4)
12. [Type XI: The $C_3$ quartic](#C3)
13. [Type XII: The $C_2$ quartic](#C2)

# Introduction<a name="introduction"></a>
The following code is taken from [Vinzant's webpage](http://sites.math.washington.edu/~vinzant/research/quartics/quartictype.sage) from [her research on quartics](http://sites.math.washington.edu/~vinzant/research/quartics.html) with Plaumann and Sturmfels. It inputs a quartic polynomial and outputs (among other things) equations for its tritangents.

In [ ]:
%run Automorphism_Groups.ipynb

# Functions<a name="functions"></a>

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F,order='degrevlex')

# The following function inputs a homogeneous quartic in QQ[x,y,z],
# and outputs the 28 bitangents
def compute_bitangents(f):
    F=QQ; T.<x,y,z>=PolynomialRing(F)
    
    # Check whether the quartic is smooth
    Grad=ideal(f,diff(f,x),diff(f,y),diff(f,z))
    if not Grad.dimension()==0:
        sys.exit("Quartic is not smooth!")

    # Create a new polynomial ring with variables a,b (for the bitangent) and a0,..,a4 (TODO)
    R.<x,y,z,a,b,a0,a1,a2,a3,a4>=PolynomialRing(F)
    f0=f.base_extend(R)
    S.<a,b>=PolynomialRing(F)
    digits=50
    threshold=0.000000000001
    almostzero=threshold

    # Take an abstract line with equation ax+by+z
    Line= a*x+b*y+z;
    puresquare=ideal(a0*a3^2-a1^2*a4,8*a0^2*a3-4*a0*a1*a2+a1^3,8*a1*a4^2-4*a2*a3*a4+a3^3,8*a0*a1*a4-4*a0*a2*a3+a1^2*a3,8*a0*a3*a4-4*a1*a2*a4+a1*a3^2,16*a0^2*a4+2*a0*a1*a3-4*a0*a2^2+a1^2*a2,16*a0*a4^2+2*a1*a3*a4-4*a2^2*a4+a2*a3^2);

    # Consider the resultant of the equation of the line with respect to z.
    Res=f0.resultant(Line,z)    
    Res=Res.subs(y=1)    
    phi=hom(R,S,[0,0,0,a,b,Res.coefficient({x:0}),Res.coefficient({x:1}),Res.coefficient({x:2}),Res.coefficient({x:3}),Res.coefficient({x:4})])
    
    bit1 = phi(puresquare)     
    
    I_ideal=singular.groebner(singular(bit1))
    singular.lib('solve.lib')
    VRing=singular.solve(I_ideal,digits)
    singular.set_ring(VRing)
    B1=singular("SOL")
    
    nreal1=0
    Bitangents=[]
    RealBitangents=[]
    for k in [1..len(B1)]:
        real=0;
        if ((B1[k][1].impart()).absValue()<threshold) and ((B1[k][2].impart()).absValue()<threshold): 
            real=1
            RealBitangents=RealBitangents+[(float(B1[k][1].repart())+float(B1[k][1].impart())*i)*x+(float(B1[k][2].repart())+float(B1[k][2].impart())*i)*y+z]
        nreal1=nreal1+real
        Bitangents=Bitangents+[(float(B1[k][1].repart())+float(B1[k][1].impart())*i)*x+(float(B1[k][2].repart())+float(B1[k][2].impart())*i)*y+z]
    Line=a*x+y     
    Res=f0.resultant(Line,y)    
    Res=Res.subs(z=1)   
    phi=hom(R,S,[0,0,0,a,0,Res.coefficient({x:0}),Res.coefficient({x:1}),Res.coefficient({x:2}),Res.coefficient({x:3}),Res.coefficient({x:4})])  
    bit2=phi(puresquare)+ideal(b)
    
    if dimension(bit2)==-1: nreal2=0
    else: 
          I_ideal=singular.groebner(singular(bit2))
          singular.lib('solve.lib')
          VRing=singular.solve(I_ideal,digits)
          singular.set_ring(VRing)
          B2=singular("SOL")
          nreal2=0
          for k in [1..len(B2)]:
                real=0
                if ((B2[k][1].impart()).absValue()<threshold) and ((B2[k][2].impart()).absValue()<threshold):
                    real=1
                    RealBitangents=RealBitangents+[(float(B2[k][1].repart())+float(B2[k][1].impart())*i)*x+y]
                nreal2=nreal2+real  
                Bitangents=Bitangents+[(float(B2[k][1].repart())+float(B2[k][1].impart())*i)*x+y]
    
    Res=f0.resultant(x)
    Res=Res.subs(z=1)
    phi=hom(R,F,[0,0,0,0,0,Res.coefficient({y:0}),Res.coefficient({y:1}),Res.coefficient({y:2}),Res.coefficient({y:3}),Res.coefficient({y:4})])  
    bit3=phi(puresquare)
    if bit3==ideal(0):
        nreal3=1
        Bitangents=Bitangents+[x]
        RealBitangents=RealBitangents+[x]
    else: nreal3=0
    
    NRealBit=nreal1+nreal2+nreal3
    if len(Bitangents)!=28:
        return OSError("Something has gone wrong. We found "+str(len(Bitangents))+" bitangents")
    # print("The quartic has 28 bitangets, stored in 'Bitangents', and "+str(NRealBit)+" real bitangents, stored in 'RealBitangents'.")
    
    return Bitangents

In [ ]:
from sage.symbolic.expression_conversions import polynomial

# Round complex numbers
def round_cx(z0,n):
    return float(round(z0.real(),n)) + float(round(z0.imag(),n))*I

# The following inputs a line ax+by+cz and outputs the tuple of coefficients [a,b,c]
# Note that line.coefficients() has errors, in that a line line 3y+2z will output [3,2] rather than [0,3,2] which is what we want
def line_to_coeffs(bitangent):
    return [CC(bitangent.coefficient({x:1})), CC(bitangent.coefficient({y:1})),CC(bitangent.coefficient({z:1}))]

# Turn a list of three coefficients [a,b,c] into the polynomial ax+by+cz=0
def coeffs_to_line(list_of_coeffs):
    return x*list_of_coeffs[0] + y*list_of_coeffs[1] + z*list_of_coeffs[2]

# Provide a normalized version of the bitangent -- old (working) version
# def normalize(bitangent):
#     b = bitangent.change_ring(ComplexField(30))
#     if b.coefficients()[0] != 0:
#         c = b*(1/b.coefficients()[0])
#         if len(c.coefficients())==3:
#             new_coeffs = [round_cx(a,5) for a in c.coefficients()]
#             c = new_coeffs[0]*x + new_coeffs[1]*y + new_coeffs[2]*z
#         return c
#     return b



# Provide a normalized version of the bitangent
def normalize(bitangent):
    # Base change to CC
    b = bitangent.change_ring(ComplexField(30))

    # If the x-coordinate is nonzero mod out by it
    if line_to_coeffs(b)[0] != 0:
        c = b*(1/line_to_coeffs(b)[0])
    # Else mod out by the y-coordinate
    elif line_to_coeffs(b)[1] != 0:
        c = b*(1/line_to_coeffs(b)[1])
    # Or the z-coordinate
    else:
        c = b*(1/line_to_coeffs(b)[2])
    
    # Round the output coordinates and return a new line
    new_coeffs = [round_cx(a,4) for a in line_to_coeffs(c)]
    return coeffs_to_line(new_coeffs)


def compute_normalized_bitangents(f):
    return [normalize(b) for b in compute_bitangents(f)]

# Apply a matrix M to a line B = {ax+by+cy=0}
def gp_action(M,B):
    vec = matrix([[B.coefficient({x:1})],[B.coefficient({y:1})],[B.coefficient({z:1})]])
    output_mat = (M*vec).transpose()
    w = list(list(output_mat)[0])
    outputline = w[0]*x + w[1]*y + w[2]*z
    return normalize(outputline)

def extract_orbit(Bitangents,list_of_indices):
    orbit = [Bitangents[j] for j in list_of_indices]
    remaining_Bitangents = [b for b in Bitangents if b not in orbit]
    return orbit,remaining_Bitangents

We write some functions to make the equations of bitangents more readable and act on them via a subgroup $G\le \text{PGL}_3(\mathbb{C})$.

In [ ]:
# We also add some commands we'll use throughout
def matrix_to_list(M):
    return [list(r) for r in M.rows()]

In [ ]:
# Generates a candidate list of the orbit of an input bitangent under a list of group elements
def candidate_orbit(bitangent,list_of_gp_elements):
    output_list = [bitangent]
    isotropy_gens = []
    for M in list_of_gp_elements:
        output_list.append(gp_action(M,bitangent))
        if gp_action(M,bitangent) == bitangent:
            isotropy_gens.append(M)
    for line in output_list:
        print(line)
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)

In [ ]:
def investigate_orbit(bitangent,group_dictionary):
    output_list = [bitangent]
    isotropy_gens = []
    # Iterate over the group
    for g in group_dictionary.keys():        
        output_list.append(gp_action(group_dictionary[g],bitangent))
        if gp_action(group_dictionary[g],bitangent) == bitangent:
            isotropy_gens.append(g)
    for line in set(output_list):
        print(line)
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)

In [ ]:
def trim(bitangent):
    x_coeff = CC(bitangent.coefficient({x:1}));
    y_coeff = CC(bitangent.coefficient({y:1}));
    z_coeff = CC(bitangent.coefficient({z:1}));
    return round_cx(x_coeff,3)*x + round_cx(y_coeff,3)*y + round_cx(z_coeff,3)*z

def investigate_orbit_Klein(bitangent,group_dictionary):
    output_list = [bitangent]
    isotropy_gens = []
    # Iterate over the group
    for g in group_dictionary.keys():        
        output_list.append(trim(gp_action(group_dictionary[g],bitangent)))
        if trim(gp_action(group_dictionary[g],bitangent)) == trim(bitangent):
            isotropy_gens.append([g,group_dictionary[g]])

    # Output the orbits
    for line in set(output_list):
        print(line)

    # Output the isotropy
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)

In [ ]:
############

def investigate_orbit_V(bitangent,group_dictionary):
    output_list = [bitangent]
    isotropy_gens = []
    # Iterate over the group
    for g in group_dictionary.keys():        
        output_list.append(gp_action(group_dictionary[g],bitangent))
        if gp_action(group_dictionary[g],bitangent) == bitangent:
            isotropy_gens.append([g,group_dictionary[g]])

    # Output the orbits
    for line in set(output_list):
        print(line)

    # Output the isotropy
    print('I think isotropy is generated by:\n')
    for isotropy_elt in isotropy_gens:
        print(isotropy_elt)
        # list_version = matrix_to_list(isotropy_elt)
        # # print(list_version)
        # output_list = []
        # for row in list_version:
        #     new_row = []
        #     for j in row:
        #         # every entry in the matx representation is either totally real or totally imaginary
        #         if j.real != 0:
        #             new_row.append(int(j.real()))
        #         elif j.imag != 0:
        #             new_row.append(int(j.imag())*I)
        #         else:
        #             print('error')
        #     output_list.append(new_row)
        # print(str(output_list) + ',')

# def investigate_orbit2(bitangent,group_dictionary):
#     output_list = [bitangent]
#     isotropy_gens = []
#     # Iterate over the group
#     for g in group_dictionary.keys():        
#         output_list.append(gp_action(group_dictionary[g],bitangent))
#         if gp_action(group_dictionary[g],bitangent) == bitangent:
#             isotropy_gens.append(group_dictionary[g])
#     for line in set(output_list):
#         print(line)
#     print('I think isotropy is generated by:\n')
#     for isotropy_elt in isotropy_gens:
#         print(isotropy_elt)

In [ ]:
def bitan_difference(bitan1,bitan2):
    # Take the difference in absolute value between each of the coefficients and sum these
    return abs(CC(bitan1.coefficient({x:1})) - CC(bitan2.coefficient({x:1}))) + abs(CC(bitan1.coefficient({y:1})) - CC(bitan2.coefficient({y:1}))) + abs(CC(bitan1.coefficient({z:1})) - CC(bitan2.coefficient({z:1})))

# This should return some indicator of how far off two equations are numerically. Uncomment and run the following for instance:
# bitan1 =x + (-1.1234 - 4.9221*I)*y + (-2.9252 - 1.4087*I)*z
# bitan2 =x + (-1.1231 - 4.9225*I)*y + (-2.9254 - 1.4091*I)*z
# bitan_difference(bitan1,bitan2)

In [222]:
# Boolean to check if two bitangents agree
def same_bitangent(bitan1,bitan2, error_rate=0.01):
    if bitan_difference(bitan1,bitan2) < error_rate:
        return true
    else:
        return false

In [ ]:
def investigate_orbit(bitangent,group_dictionary):
    output_list = [bitangent]

    for g in group_dictionary.keys():
        g_bitangent = gp_action(group_dictionary[g],bitangent)

        new_bitangent = True
        for b in output_list:
            if same_bitangent(g_bitangent,b):
                new_bitangent = False

        if new_bitangent == True:
            output_list.append(g_bitangent)
    return list(set(output_list))

def investigate_isotropy(bitangent,group_dictionary):
    isotropy_gens = []
    for g in group_dictionary.keys():
        # print(g)
        g_bitangent = gp_action(group_dictionary[g],bitangent)
        # print(g_bitangent)
        if same_bitangent(g_bitangent,bitangent):
            isotropy_gens.append(g)
    return isotropy_gens

We want to compute all the orbits of the bitangents for the [cubic surfaces found here](https://mat.uab.cat/~francesc/mates/autgen3.pdf).b

TODO -- FIX EVERYTHING ABOVE HERE/RENAME FUNCTIONS/ETC

# Type I: The Klein quartic<a name="klein"></a>
Our first quartic is the *Klein quartic*, defined by the equation
$$x^3y + y^3z + z^3x$$
This has the largest possible automorphism group, encoded as `PSL_dict` here.

It is a classical fact that the automorphism group of the Klein quartic acts transitively on its 28 bitangents, however we can also observe this computationally. We first compute a list of all the bitangents of the Klein quartic.

In [ ]:
Klein = x^3*y + y^3*z + z^3*x;
Klein_Bitangents = compute_normalized_bitangents(Klein);Klein_Bitangents

We can look at the first bitangent `Klein_Bitangents[0]`, and try to examine what its orbit is under the action of the automorphism group using our `investigate_orbit` command:

In [ ]:
investigate_orbit(Klein_Bitangents[0],PSL_dict)

It is clear to see that we get all 28 bitangents in the orbit of the first one under the action of $\text{PSL}_2(7)$. It remains to ask what the isotropy group of a single bitangent is --- since the action is transitive, we should obtain an index 48 subgroup in $\text{PSL}_2(7)$.

In [ ]:
investigate_isotropy(compute_normalized_bitangents(Klein)[0],PSL_dict)

We can then return to GAP and verify that these matrices generate a subgroup of $\text{PSL}_2(7)$ which is isomorphic to $S_3$. Therefore as a $\text{PSL}_2(7)$-set, the 28 bitangents of the Klein quartic are in equivariant bijection with the left cosets $[\text{PSL}_2(7)/S_3]$. As a sanity check, $|\text{PSL}_2(7)| = 168$ and $|S_3| = 6$, and $168/6 = 28$.

# Type II: Fermat<a href="fermat"></a>

## (II) Bitangents

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
Fermat = x^4 + y^4 + z^4

Fermat_Bitangents = compute_normalized_bitangents(Fermat);Fermat_Bitangents

Again, we can look at the orbit of the first bitangent:

In [ ]:
investigate_orbit(Fermat_Bitangents[0],Fermat_dict)

Hence we get 12 bitangents in this orbit. In terms of the indices of the list `Fermat_Bitangents`, these are the entries `[0,1,2,3,24,25,26,27,8,9,10,11]`. Again we can investigate the isotropy:

In [ ]:
investigate_isotropy(Fermat_Bitangents[0],Fermat_dict)

Going back to GAP, we get a copy of $C_8$ living inside the automorphism group of the Fermat quartic. Hence the 12 bitangents above correspond to the transitive set $[C_4^{\times 2}\rtimes S_3 / C_8]$.

We can look at the remaining bitangents, for instance the element `Fermat_Bitangents[4]` which was not in the orbit above:

In [ ]:
investigate_orbit(Fermat_Bitangents[4],Fermat_dict)

These are the remaining 16 bitangents -- their indices in `Fermat_Bitangents` are `[16,6,20,21,15,12,14,17,18,7,4,5,23,22,13,19]`. Again we study isotropy:

In [ ]:
investigate_isotropy(Fermat_Bitangents[4],Fermat_dict)

This gives us a copy of $C_6$ inside of $C_4^{\times 2}\rtimes S_3$. Hence altogether we get an equivariant bijection
$$\text{28 bitangen

# Type III: The $(48,33)$ quartic<a name="type3"></a>
There is a unique quartic of Type III, defined by the equation
$$x^4 + y^4 + z^4 + (4\rho+2)x^2y^2,$$
where $\rho$ is a primitive 3rd root of unity (note that $4\rho + 2 = 2\sqrt{-3}$, which is another common form for this quartic). Its automorphism group has order 48, and GAP ID `[48,33]`. It is accessible via `GpIII_Dict`.



## (III) Bitangents

We would like to compute equations for its bitangents, however since $\rho\notin\mathbb{Q}$, we must modify the function `compute_bitangents`. To that end we include the following

In [ ]:
def compute_bitangents_ideal_Q3(f):
    F=CyclotomicField(3); T.<x,y,z>=PolynomialRing(F)
    # Check whether the quartic is smooth
    # Grad=ideal(f,diff(f,x),diff(f,y),diff(f,z))
    # if not Grad.dimension()==0:
    #     sys.exit("Quartic is not smooth!")
    f = T(f)
    R.<x,y,z,a,b,a0,a1,a2,a3,a4>=PolynomialRing(F)
    f0=f.base_extend(R)
    S.<a,b>=PolynomialRing(F)
    digits=50
    threshold=0.000000000001
    almostzero=threshold

    # Make a formal line
    Line= a*x+b*y+z;
    
    puresquare=ideal(a0*a3^2-a1^2*a4,8*a0^2*a3-4*a0*a1*a2+a1^3,8*a1*a4^2-4*a2*a3*a4+a3^3,8*a0*a1*a4-4*a0*a2*a3+a1^2*a3,8*a0*a3*a4-4*a1*a2*a4+a1*a3^2,16*a0^2*a4+2*a0*a1*a3-4*a0*a2^2+a1^2*a2,16*a0*a4^2+2*a1*a3*a4-4*a2^2*a4+a2*a3^2);
    Res=f0.resultant(Line,z)    
    Res=Res.subs(y=1)    
    phi=hom(R,S,[0,0,0,a,b,Res.coefficient({x:0}),Res.coefficient({x:1}),Res.coefficient({x:2}),Res.coefficient({x:3}),Res.coefficient({x:4})])
    bit1 = phi(puresquare)    
    
    I_ideal=singular.groebner(singular(bit1))
    return I_ideal

Dolg = x^4 + y^4 + z^4 + 2*sqrt(-3)*(x^2)*(y^2)
F=CyclotomicField(12); T.<x,y,z>=PolynomialRing(F)

# Grad = ideal(Dolg,diff(Dolg,x),diff(Dolg,y),diff(Dolg,z))
Iideal = compute_bitangents_ideal_Q3(Dolg);Iideal
print(Iideal)

This gives us an ideal in a polynomial ring over $\mathbb{Q}(\zeta_3)$ that we wish to solve -- the solutions will be bitangents of the form $ax+by+z=0$.

In [ ]:
# Grad.dimension()
# cyc

R.<a,b> =PolynomialRing(CC)
rho = exp(2*pi*i/3)
a,b = var('a b')
f0 = a^4+(-4*rho-2)*a^2*b^2+b^4+4
f1 = 3*a^3*b^3+(2*rho+1)*a*b^5+(4*rho+2)*a*b
f2 = 7*a^2*b^5+(2*rho+1)*b^7+2*a^2*b+(8*rho+4)*b^3
f3 = 2*a*b^7+(2*rho+1)*a^3*b+a*b^3
f4 = 2*b^9-5*a^4*b+(-6*rho-3)*a^2*b^3+10*b^5+8*b


candidates = solve([f1==0,f2==0,f3==0,f0==0,f4==0],a,b); candidates

This gives us 24 bitangents. To obtain the other 4, we need to consider the case where $z=0$..... TODO

In [ ]:
QuarticIIIBis = [x + (-0.50000084 + 0.50000314*I)*z,
x + (-0.50000314 - 0.50000084*I)*z,
x - y + (0.500000000000000 - 0.866030000000000*I)*z,
x - y + (-0.500000000000000 + 0.866030000000000*I)*z,
x - y + (-0.866030000000000 - 0.500000000000000*I)*z,
x - y + (0.866030000000000 + 0.500000000000000*I)*z,
x + (0.50000314 + 0.50000084*I)*z,
x + (0.50000084 - 0.50000314*I)*z,
y + (0.50000314 + 0.50000084*I)*z,
y + (-0.50000084 + 0.50000314*I)*z,
y + (0.50000084 - 0.50000314*I)*z,
y + (-0.50000314 - 0.50000084*I)*z,
x + 1.00000000000000*I*y + (0.500000000000000 + 0.866030000000000*I)*z,
x + 1.00000000000000*I*y + (0.866030000000000 - 0.500000000000000*I)*z,
x + 1.00000000000000*I*y + (-0.866030000000000 + 0.500000000000000*I)*z,
x + 1.00000000000000*I*y + (-0.500000000000000 - 0.866030000000000*I)*z,
x + (-1.00000000000000*I)*y + (0.500000000000000 + 0.866030000000000*I)*z,
x + (-1.00000000000000*I)*y + (0.866030000000000 - 0.500000000000000*I)*z,
x + (-1.00000000000000*I)*y + (-0.866030000000000 + 0.500000000000000*I)*z,
x + (-1.00000000000000*I)*y + (-0.500000000000000 - 0.866030000000000*I)*z,
x + y + (0.500000000000000 - 0.866030000000000*I)*z,
x + y + (-0.866030000000000 - 0.500000000000000*I)*z,
x + y + (-0.500000000000000 + 0.866030000000000*I)*z,
x + y + (0.866030000000000 + 0.500000000000000*I)*z,
x + (-0.36602540 - 0.36602540*I)*y,
x + (0.36602540 + 0.36602540*I)*y,
x + (-1.3660254 + 1.3660254*I)*y,
x + (1.3660254 - 1.3660254*I)*y]


In [ ]:
investigate_orbit(QuarticIIIBis[0],GpIII_Dict)

In [ ]:
investigate_isotropy(QuarticIIIBis[0],GpIII_Dict)

In [ ]:
investigate_orbit(QuarticIIIBis[27],GpIII_Dict)

In [ ]:
investigate_isotropy(QuarticIIIBis[27],GpIII_Dict)

# Type IV: The $S_4$ quartic<a name="S4"></a>
For the group $S_4$, we obtain a pencil of *octahedral quartics*, given by the equation
 $$X^4 + Y^4 + Z^4 + a(X^2 Y^2 + Y^2 Z^2 + Z^2 X^2),$$
for any $a\in \mathbb{C}$. Here $S_4$ is subconjugate to $\text{PSL}_2(7)$ as well as $C_4^{\times 2}\rtimes S_3$ inside $\text{PGL}_3(\mathbb{C})$, reflected by certain values of $a$ which allow us to exhibit the Klein and Fermat quartics in this pencil.

We pick $a=-3$ (not one of these special values) to carry out our computation:

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F) # redefine the base field back to QQ

S4_Quartic = x^4 + y^4 + z^4 -3*(x^2*y^2 + y^2*z^2 + z^2*x^2)
S4_Bitangents = compute_normalized_bitangents(S4_Quartic);S4_Bitangents
# S4_Bitangents = [normalize(b) for b in S4_Bitangents];S4_Bitangents

We end up seeing three orbits, with isotropy groups given by the even and odd copies of $C_2$ in $S_4$, and $S_3$, respectively.

## $C_2^o$ isotropy

In [ ]:
investigate_orbit(S4_Bitangents[0],S4_dict)

In the list `S4_Bitangents` these are the 12 bitangents with indices `[0,1,2,3,18,19,26,27,25,24,16,17]`.

In [ ]:
investigate_isotropy(S4_Bitangents[0],S4_dict)

## $C_2^e$ isotropy

In [ ]:
investigate_orbit(S4_Bitangents[4],S4_dict)

These are `[10,4,5,6,11,9,12,20,23,15,8,7]` in `S4_Bitangents`.

In [ ]:
investigate_isotropy(S4_Bitangents[4],S4_dict)

## $S_3$ isotropy

Finally we have four remaining bitangents, which we will see all lie in an orbit and therefore must have $S_3$ isotropy.

In [ ]:
investigate_orbit(S4_Bitangents[13],S4_dict)

In the list `S4_Bitangents` these are the 12 bitangents with indices `[13,14,21,22]`.

In [ ]:
investigate_isotropy(S4_Bitangents[13],S4_dict)

# Type V: The $(16,13)$ quartic<a name="1613"></a>
For Type V, we obtain a pencil of quartics with equations
$$X^4 + Y^4 + Z^4 + aX^2 Y^2,$$
and symmetry given by the *Pauli group* $P = C_4\circledcirc K_4$. We pick a generic value $a=-4$ to obtain a Type V quartic with no additional symmetry beyond $P$:

In [ ]:
# F=QQ; T.<x,y,z>=PolynomialRing(F)
V_Quartic = x^4 + y^4 + z^4 - 4*x^2*y^2
V_Bitangents = compute_normalized_bitangents(V_Quartic);V_Bitangents

## $C_2^{(1)}$ isotropy

We obtain one orbit with isotropy $C_2$ as follows:

In [ ]:
investigate_orbit(V_Bitangents[0],VGrp_dict)

This gives us eight bitangents, with indices `[0,1,2,3,20,21,22,23]` in `V_Bitangents`. Isotropy is given by the following:

In [ ]:
investigate_isotropy(V_Bitangents[0],VGrp_dict)

## $C_2^{(2)}$ isotropy
We obtain another orbit of bitangents with isotropy $C_2$, however this is a non-conjugate copy of $C_2$ in the Pauli group:

In [ ]:
investigate_orbit(V_Bitangents[4],VGrp_dict)

This gives us eight bitangents, with indices `[4,5,6,7,16,17,18,19]`, and isotropy

In [ ]:
investigate_isotropy(V_Bitangents[4],VGrp_dict)

## $C_2^{(3)}$ isotropy

We have yet another orbit of eight bitangents, whose isotropy is another non-conjugate copy of $C_2$ in the Pauli group. These have indices `[8,9,10,11,12,13,14,15]`

In [ ]:
investigate_orbit(V_Bitangents[8],VGrp_dict)

In [ ]:
investigate_isotropy(V_Bitangents[8],VGrp_dict)

## $C_4^Z$ isotropy

The remaining four bitangents, with indices `[24,25,26,27]` form an orbit with isotropy a cyclic subgroup of order four, characterized by the property that it is central in the Pauli group.

In [ ]:
investigate_orbit(V_Bitangents[24],VGrp_dict)

In [ ]:
investigate_isotropy(V_Bitangents[24],VGrp_dict)

# Type VI: The $C_9$ quartic<a name="C9"></a>
There is a unique planar quartic with symmetry group cyclic of order 9, given by the equation
$$X^4 + XY^3 + YZ^3.$$

In [ ]:
C9_Quartic = x^4 + x*y^3 + y*z^3
C9_Bitangents = compute_normalized_bitangents(C9_Quartic);C9_Bitangents

## Trivial isotropy

We have three orbits with the same isotropy, given by the indices

- `[4,0,23,24,25,3,26,8,7]`
- `[6,1,5,20,12,21,22,19,11]`
- `[2,9,10,13,14,15,16,17,18]`

In [ ]:
investigate_orbit(C9_Bitangents[0],C9_dict)

In [ ]:
investigate_orbit(C9_Bitangents[1],C9_dict)

In [ ]:
investigate_orbit(C9_Bitangents[2],C9_dict)

We observe they all have trivial isotropy

In [ ]:
investigate_isotropy(C9_Bitangents[0],C9_dict)

In [ ]:
investigate_isotropy(C9_Bitangents[1],C9_dict)

In [ ]:
investigate_isotropy(C9_Bitangents[2],C9_dict)

## $C_9$ isotropy

We have accounted for 27 of our 28 bitangents, implying the remaining one must be fixed by the action of $C_9$. We can verify this:

In [ ]:
investigate_isotropy(C9_Bitangents[27],C9_dict)

# Type VII: The $D_8$ quartic<a name="D8"></a>
We have a two-dimensional family of $D_8$-symmetric quartic with equations
$$X^4 + Y^4 + Z^4 + aX^2 Y^2 + bXYZ^2.$$
We pick the special values $a=-3$ and $b=1$ to investigate the action of $D_8$ on the bitangents.

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
D4_Quartic = x^4 + y^4 + z^4 - 3*x^2*y^2 + x*y*z^2
D4_Bitangents = compute_normalized_bitangents(D4_Quartic);D4_Bitangents

We have the following isotropies:

- $[D_8/e]$ is `[2,1,13,12,0,3,14,15]`
- $[D_8/C_2]$ has four orbits `[4,7,20,23],[22,6,5,21],[19,9,8,18],[10,16,17,11]`
- $[D_8/C_2^Z]$ is `[24,25,26,27]`,
here $C_2^Z \le D_8$ denotes the center subgroup, and $C_2$ without a decoration denotes the non-center conjugacy subgroup which is cyclic of order two

# Type VIII: $C_6$<a name="C6"></a>
There is a pencil of $C_6$-symmetric quartics, given by the equation
$$ x^4 + y^4 + ax^2 y^2 + yz^3$$
For $a=-3$, we compute the bitangents as follows:

In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F)
C6_Quartic = z^3*y +x^4 -3*x^2*y^2 +y^4
C6_Bitangents = compute_normalized_bitangents(C6_Quartic);C6_Bitangents

There are some rounding errors so we edit this set by hand:

In [ ]:
C6_Bitangents=[y + (-0.92831777)*z,
 y + (0.46415888 + 0.80394677*I)*z,
 y + (0.46415888 - 0.80394677*I)*z,
 x + 1.85047*y + (-0.39283)*z,
 x + (-1.85047)*y + 0.39283*z,
 x + 0.56125*I*y + 1.18243*I*z,
 x + (-0.56125*I)*y + (-1.18243*I)*z,
 x + (0.70756 - 0.23507*I)*y + (-0.52306 - 0.12198*I)*z,
 x + (-0.70756 + 0.23507*I)*y + (0.52306 + 0.12198*I)*z,
 x + (0.70756 + 0.23507*I)*y + (-0.52306 + 0.12198*I)*z,
 x + (-0.70756 - 0.23507*I)*y + (0.52306 - 0.12198*I)*z,
 x + (-0.56125*I)*y + (-1.02401 + 0.59121*I)*z,
 x + 0.56125*I*y + (1.02401 - 0.59121*I)*z,
 x + 0.56125*I*y + (-1.02401 - 0.59121*I)*z,
 x + (-0.56125*I)*y + (1.02401 + 0.59121*I)*z,
 x + (-0.70756 - 0.23507*I)*y + (-0.15589 + 0.51397*I)*z,
 x + (0.70756 + 0.23507*I)*y + (0.15589 - 0.51397*I)*z,
 x + (-0.70756 + 0.23507*I)*y + (-0.15589 - 0.51397*I)*z,
 x + (0.70756 - 0.23507*I)*y + (0.15589 + 0.51397*I)*z,
 x + (-0.70756 + 0.23507*I)*y + (-0.36717 + 0.39199*I)*z,
 x + (0.70756 - 0.23507*I)*y + (0.36717 - 0.39199*I)*z,
 x + (-0.70756 - 0.23507*I)*y + (-0.36717 - 0.39199*I)*z,
 x + (0.70756 + 0.23507*I)*y + (0.36717 + 0.39199*I)*z,
 x + (-1.85047)*y + (-0.19641 - 0.3402*I)*z,
 x + 1.85047*y + (0.19641 + 0.3402*I)*z,
 x + (-1.85047)*y + (-0.19641 + 0.3402*I)*z,
 x + 1.85047*y + (0.19641 - 0.3402*I)*z,
 y]

We have the following isotropies:

- $[C_6/C_2]$ is `[0,1,2]`
- $[C_6/e]$ - there are four orbits of this type: `[24,26,3,25,4,23]`, `[6,11,14,13,12,5]`, `[7,20,18,19,8,17]`, and `[16,9,10,22,15,21]]`.

# Type IX: The $S_3$ quartic<a name="S3"></a>
We have a 2D family of $S_3$-symmetric quartics, with equations
$$X^3Z + Y^3 Z + X^2 Y^2 + aXYZ^2 + bZ^4.$$
We pick the values $a=-25$ and $b=10$ to compute the bitangents:

In [ ]:
S3_Quartic_affine = x^3 + y^3 + x^2 * y^2 - 25*x*y + 10
S3_Bitangents = compute_normalized_bitangents(S3_Quartic);S3_Bitangents

For isotropy we have
- A single fixed bitangent `[5]`
- Three orbits of the form $[S_3/C_2]$: `[18,19,3]`, `[12,24,17]`, and `[16,25,13]`
- Three orbits of the form $[S_3/e]$: `[1,23,22,26,27,0]`, `[14,21,15,20,2,4]`, and `[6,10,11,8,7,9]`

# Type X: The $K_4$ quartic<a name="K4"></a>
We have a 3D family of $K_4$-symmetric quartics, given by $$X^4 + Y^4 + Z^4 + aX^2 Y^2 + bY^2Z^2 + c Z^2 X^2$$
with action given by the diagonal matrices $$\text{diag}[-1,1,1] \qquad \text{and}\qquad \text{diag}[1,-1,1].$$
We choose to call these the *left* and *right* copies of $C_2$ in $K_4$.

In [ ]:
left_generator = matrix([[-1,0,0],[0,1,0],[0,0,1]])
right_generator = matrix([[1,0,0],[0,-1,0],[0,0,1]])
diagonal_generator = matrix([[-1,0,0],[0,-1,0],[0,0,1]])

We pick the special values $(a,b,c) = (-9,-3,-8)$ to compute the bitangents

In [ ]:
K4_Quartic = x^4 + y^4 + z^4 -9*x^2*y^2 - 3*y^2*z^2 - 8*x^2*z^2

K4_Bitangents = compute_normalized_bitangents(K4_Quartic);K4_Bitangents

Isotropy is:
- $[K_4/C_2^L]$: `[0,1]`, `[2,3]`
- $[K_4/e]$: `[4,5,18,19]`, `[6,7,16,17]`, `[8,9,14,15]`, `[10,11,12,13]`
- $[K_4/C_2^R]$: `[20,21]`, `[22,23]`
- $[K_4/C_2^\Delta]$: `[24,25]`, `[26,27]`

# Type XI: The $C_3$ quartic<a name="C3"></a>
We have a 3D family of $C_3$-symmetric quartics given by the equations:
$$x(x-y)(x-ay)(x-by) + yz^3.$$
The action of $C_3$ occurs via the diagonal matrix $\begin{pmatrix} 1 & 0 & 0\\ 0& 1 &0 \\ 0 & 0 & \zeta_3\end{pmatrix}$ where $\zeta_3$ is a primitive 3rd root of unity.

We pick the special values $a=2$ and $b=3$ to compute bitangents:

In [ ]:
C3_Quartic = z^3*y + x*(x-y)*(x-2*y)*(x-3*y)
C3_Bitangents = compute_normalized_bitangents(C3_Quartic);C3_Bitangents

The istropies are:
- A single fixed bitangent, the last one `[27]`
- Free transitive orbits $[C_3/e]$ given by the following indices: `[0,25,26]`, `[1,15,16]`, `[2,11,12]`, `[3,18,23]`, `[4,17,24]`,`[5,14,19]`, `[6,13,20]`, `[7,10,21]`, `[8,9,22]`.

# Type XII: The $C_2$ quartic
Finally we have a 4D family of $C_2$-symmetric quartics, given by the equations
$$x^4 + x^2 L_2 + L_4,$$
where $L_n$ is any binary form of degree $n$ in $y$ and $z$. Here the $C_2$-action comes from sending $x\mapsto -x$. We can easily see the action on bitangents by swapping signs. We pick the special values
$$L_2 = -y^2 - 4z^2\quad \text{ and } \quad L_4 = y^4 - y^2 z^2 - z^4$$

In [ ]:
L2 = -y^2 - 4*z^2
L4 = y^4 - y^2*z^2 - z^4
C2_Quartic = x^4 + x^2*L2 + L4
C2_Bitangents = compute_normalized_bitangents(C2_Quartic);C2_Bitangents

For isotropy:
- The first four bitangents are clearly fixed, as they have no $x$-coordinate.
- The remaining 24 bitangents come in pairs under the $C_2$-action: `[4,5]`, `[6,7]`,...`[26,27]`